# Wine Quality e Type
Questo notebook illustra il processo di costruzione, addestramento e valutazione di un modello di machine learning per la previsione della qualità del vino. In particolare si addestrerà una rete neurale multi input (N input pari al numero di colonne), multi output (2 colonne target, Qualità - valore numerico che indica la qualità del vino con una scala 1-10, e Tipo - variabile binaria rosso o bianco)
Il focus principale è la creazione di un modello di deep learning per prevedere la qualità del vino basandosi su varie caratteristiche chimiche.

- Importazione delle librerie necessarie, tra cui TensorFlow per la costruzione del modello, pandas e seaborn per la manipolazione e la visualizzazione dei dati, e scikit-learn per la divisione del dataset
- Lettura del dataset
- Visualizzazione delle qualità uniche del vino presenti nel dataset

In [3]:
#Importing the libraries. 

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import TensorBoard
import tensorboard

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np

from tensorflow.python.keras.utils.vis_utils import plot_model

from scipy.stats import norm
from scipy import stats

from sklearn.model_selection import train_test_split

c:\Users\pagan133\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
df = pd.read_csv("./dataset/winequalityN.csv")

df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
distinct_qualities = df['quality'].unique()
display(distinct_qualities)

array([6, 5, 7, 8, 4, 3, 9], dtype=int64)

In [0]:
#Converting string column to categorical numeric. 
df['type'] = df['type'].apply(lambda x:0 if (x == 'white') else 1)

In [0]:
#Get the numbers of Null values in columns, in descending order. 
df.isna().sum().sort_values(ascending=False).head(9)


fixed acidity          10
pH                      9
volatile acidity        8
sulphates               4
citric acid             3
residual sugar          2
chlorides               2
type                    0
free sulfur dioxide     0
dtype: int64


Viene creato un nuovo dataset eliminando le righe che contengono valori nulli o vuoti. Questa operazione è necessaria per assicurarsi che il dataset sia pulito e non contenga dati mancanti, in modo da poter eseguire analisi e modellazione accurata.

Viene controllato se ci sono ancora valori nulli nel dataset. Viene calcolato il numero di valori nulli per ciascuna colonna, ordinati in ordine decrescente e vengono mostrati solo i primi 8. Questa operazione è utile per identificare eventuali colonne con valori mancanti e valutare l'impatto che potrebbero avere sull'analisi o sul modello.


In [0]:
#Creating the new dataset without null, or empty, values. 
df = df.dropna(axis=0)

In [0]:
#Check if we have more nulls. 
df.isna().sum().sort_values(ascending=False).head(8)

type                    0
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
dtype: int64


Viene creato il set di dati di addestramento e di test. Il 80% dei dati viene utilizzato per l'addestramento e il 20% per il test. Questa divisione è comune per valutare le prestazioni del modello su dati non visti durante l'addestramento.

Viene quindi creato un ulteriore set di dati di addestramento e di validazione. Il 20% dei dati di addestramento viene utilizzato per la validazione. Questa divisione è utile per valutare le prestazioni del modello durante l'addestramento e per evitare l'overfitting.

Viene definita una funzione che estrae le etichette "type" e "quality" dal dataset. Queste etichette vengono rimosse dal dataset originale e convertite in array numpy. Questa operazione è necessaria per preparare le etichette da utilizzare come output del modello di machine learning.

In [0]:
#Create the train and test data. 80% tain and 20 data. 
train, test = train_test_split(df, test_size=0.2, random_state = 1)

#From the train Data we are going to get a 20% more to create the validation data. 
train, val = train_test_split(train, test_size=0.2, random_state = 1)


In [0]:
#With this function we got the labels *type* and *quality*,  ready to pass to the Model. 
def get_labels(df):
    type_wine = df.pop('type')
    type_wine = np.array(type_wine)
    quality = df.pop('quality')
    quality = np.array(quality)
    return (quality, type_wine)

In [0]:
#Getting the labels for train, test and validate. 
train_y = get_labels(train)
test_y = get_labels(test)
val_y = get_labels(val)

In [0]:
#Get Mean and std but only from training data. 
train_stats = train.describe()
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
fixed acidity,4136.0,7.196905,1.296497,3.80000,6.4000,7.000000,7.62500,15.90000
volatile acidity,4136.0,0.337378,0.163193,0.08000,0.2200,0.290000,0.40000,1.58000
citric acid,4136.0,0.319108,0.144535,0.00000,0.2500,0.310000,0.39000,1.66000
residual sugar,4136.0,5.510638,4.812892,0.60000,1.8000,3.200000,8.20000,65.80000
chlorides,4136.0,0.055627,0.035004,0.01200,0.0380,0.047000,0.06400,0.61100
free sulfur dioxide,4136.0,30.732834,17.917700,1.00000,17.0000,29.000000,42.00000,289.00000
total sulfur dioxide,4136.0,116.689313,56.244277,7.00000,80.0000,119.000000,156.00000,440.00000
density,4136.0,0.994708,0.003034,0.98713,0.9923,0.994885,0.99704,1.03898
pH,4136.0,3.218873,0.160356,2.72000,3.1100,3.210000,3.32000,4.01000
sulphates,4136.0,0.530771,0.148288,0.23000,0.4300,0.510000,0.60000,2.00000


In [0]:
# Normalize the data, but with the mean and std of only train data. 
def scale_data(df):
    return (df - train_stats['mean']) / train_stats['std']

#Scaling the 3 datasets. 
train_X = scale_data(train)
test_X = scale_data(test)
val_X = scale_data(val)


La rete neurale implementata è un modello di regressione multi-output. È stata progettata in questo modo per affrontare un problema di previsione di due variabili di output: il tipo di vino (variabile binaria) e la qualità del vino (variabile continua).

Il modello è composto da un input layer che specifica la forma dei dati in ingresso, seguito da due dense layers con funzione di attivazione ReLU. Questi layer sono comuni a entrambe le variabili di output.

Per la variabile di output "tipo di vino", è stato aggiunto un output layer con funzione di attivazione sigmoide. Questo layer restituisce una probabilità che il vino sia di un determinato tipo.

Per la variabile di output "qualità del vino", è stato aggiunto un dense layer aggiuntivo chiamato "quality_layer" con funzione di attivazione ReLU. Questo layer introduce una diversificazione nel modello.

Infine, sono stati definiti due output layers: "y_q_layer" per la variabile di output "qualità del vino" e "y_t_layer" per la variabile di output "tipo di vino".

Il modello è stato compilato utilizzando l'ottimizzatore Adam e le seguenti funzioni di loss e metriche:
- Per la variabile di output "tipo di vino", è stata utilizzata la funzione di loss "binary_crossentropy" e la metrica "accuracy".
- Per la variabile di output "qualità del vino", è stata utilizzata la funzione di loss "mse" (Mean Squared Error) e la metrica "RootMeanSquaredError".

Questo tipo di rete neurale è stato scelto per gestire un problema di previsione multi-output, in cui si desidera prevedere più di una variabile di output. L'aggiunta del dense layer "quality_layer" permette di introdurre una diversificazione nel modello, consentendo di catturare relazioni più complesse tra le variabili di input e l'output "qualità del vino".

Un hidden layer, in particolare di tipo dense, è uno strato di neuroni in una rete neurale che riceve input dai neuroni dello strato precedente e invia output ai neuroni dello strato successivo. Questo strato è chiamato "hidden" perché i suoi neuroni non sono direttamente visibili all'esterno della rete neurale.

Nel contesto di una rete neurale densamente connessa (dense neural network), ogni neurone in uno strato hidden dense è connesso a tutti i neuroni dello strato precedente e a tutti i neuroni dello strato successivo. Questo significa che ogni neurone riceve input da tutti i neuroni dello strato precedente e invia output a tutti i neuroni dello strato successivo.

La funzione di attivazione, in particolare la funzione ReLU (Rectified Linear Unit), è una funzione matematica che viene applicata ai valori di output dei neuroni in uno strato hidden dense. La funzione ReLU è definita come f(x) = max(0, x), dove x è il valore di input del neurone. Questa funzione è non lineare e introduce la non linearità nella rete neurale.

La funzione ReLU è ampiamente utilizzata nelle reti neurali perché è semplice da calcolare e risolve il problema della scomparsa del gradiente. Inoltre, la funzione ReLU è in grado di approssimare funzioni complesse e di introdurre la capacità di apprendimento non lineare nella rete neurale.

In sintesi, uno hidden layer di tipo dense è uno strato di neuroni in una rete neurale che riceve input dai neuroni dello strato precedente e invia output ai neuroni dello strato successivo. La funzione di attivazione ReLU viene applicata ai valori di output dei neuroni in questo strato per introdurre non linearità nella rete neurale e consentire l'apprendimento di relazioni complesse tra i dati di input e l'output desiderato.


Le due funzioni di loss utilizzate nel modello sono "binary_crossentropy" e "mse" (Mean Squared Error).

La funzione di loss "binary_crossentropy" viene utilizzata per la variabile di output "tipo di vino", che è una variabile binaria. Questa funzione di loss calcola l'errore tra la probabilità predetta dal modello per il vino di un determinato tipo e il valore reale della variabile di output. La formula della funzione di loss "binary_crossentropy" è:

loss = - (y_true * log(y_pred) + (1 - y_true) * log(1 - y_pred))

Dove:
- y_true è il valore reale della variabile di output (0 o 1)
- y_pred è la probabilità predetta dal modello per il vino di un determinato tipo

La funzione di loss "mse" viene utilizzata per la variabile di output "qualità del vino", che è una variabile continua. Questa funzione di loss calcola l'errore quadratico medio tra il valore predetto dal modello per la qualità del vino e il valore reale della variabile di output. La formula della funzione di loss "mse" è:

loss = (1/n) * sum((y_true - y_pred)^2)

Dove:
- y_true è il valore reale della variabile di output
- y_pred è il valore predetto dal modello per la qualità del vino
- n è il numero di campioni nel dataset

Le due funzioni di metrica utilizzate nel modello sono "accuracy" e "RootMeanSquaredError".

La metrica "accuracy" viene utilizzata per valutare le prestazioni del modello nella previsione del tipo di vino. Questa metrica calcola la percentuale di predizioni corrette rispetto al numero totale di predizioni. La formula della metrica "accuracy" è:

accuracy = (numero di predizioni corrette) / (numero totale di predizioni)

La metrica "RootMeanSquaredError" viene utilizzata per valutare le prestazioni del modello nella previsione della qualità del vino. Questa metrica calcola la radice quadrata dell'errore quadratico medio tra il valore predetto dal modello e il valore reale della variabile di output. La formula della metrica "RootMeanSquaredError" è:

RMSE = sqrt((1/n) * sum((y_true - y_pred)^2))

Dove:
- y_true è il valore reale della variabile di output
- y_pred è il valore predetto dal modello per la qualità del vino
- n è il numero di campioni nel dataset

In [0]:
#Start with the input layer, where we must indicate the shape of the Data passed to the model. 
inputs = tf.keras.layers.Input(shape=(11,))

#Add dense layers to the input layer. These layers are commom to both predicted variables. 
x = Dense(units=32, activation='relu')(inputs)
x = Dense(units=32, activation='relu')(x)

#Add the output layer for the Wine type using Sigmoid activation. 
y_t_layer = Dense(units = 1, activation='sigmoid', name='y_t_layer')(x)

#Here we diversificate the model adding a new Dense layer to the Base layers (x)
quality_layer=Dense(units=64, name='quality_layer', activation='relu')(x)

#The output layer for the quality wine variable. It's added below the Dense Layer: quality_layer 
y_q_layer = Dense(units=1, name='y_q_layer')(quality_layer)

#The Model is created indicating the inputs and outputs. 
#We have only one Input, but we can create models with multiple inputs. 
#The name in outputs is the same of the variables, and the internal name of the layer. 
model = Model(inputs=inputs, outputs=[y_q_layer, y_t_layer])

#I tested two optimizers and choosed Adam, but feel free to test yourself. 
#optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001)
optimizer = tf.keras.optimizers.Adam()

#To compile the model we use two dictionaries, to indicate the loss functions and metrics 
#for each output layer. Note that the name of the layer must be the same than the 
#internal name of the layer. 
model.compile(optimizer=optimizer, 
              loss = {'y_t_layer' : 'binary_crossentropy', 
                      'y_q_layer' : 'mse'
                     },
              metrics = {'y_t_layer' : 'accuracy', 
                         'y_q_layer': tf.keras.metrics.RootMeanSquaredError()
                       }
             )


Le summary del modello ci forniscono informazioni sulle diverse layer del modello, inclusi il tipo di layer, la forma dell'output di ogni layer e il numero di parametri di ogni layer.

Nel caso specifico del modello in questione, la summary ci indica che ci sono 4 layer nel modello:

1. Un layer di input con una forma di (None, 11), che indica che il modello accetta un input di dimensione (batch_size, 11).
2. Due layer densi con 32 unità ciascuno e funzione di attivazione ReLU.
3. Un layer di output per la variabile "tipo di vino" con 1 unità e funzione di attivazione sigmoide.
4. Un layer denso con 64 unità e funzione di attivazione ReLU.
5. Un layer di output per la variabile "qualità del vino" con 1 unità.

Il numero di parametri di ogni layer è determinato dalla formula: (numero_di_unità_del_layer_precedente + 1) * numero_di_unità_del_layer_corrente. Il "+1" è dovuto al bias.

Quindi, nel caso del primo layer denso, abbiamo (11 + 1) * 32 = 384 parametri. Nel caso del secondo layer denso, abbiamo (32 + 1) * 32 = 1056 parametri. Nel caso del layer di output per la variabile "tipo di vino", abbiamo (32 + 1) * 1 = 33 parametri. Nel caso del layer denso successivo, abbiamo (32 + 1) * 64 = 2112 parametri. Infine, nel caso del layer di output per la variabile "qualità del vino", abbiamo (64 + 1) * 1 = 65 parametri.

Questi parametri rappresentano i pesi e i bias delle connessioni tra i neuroni dei diversi layer del modello. Sono questi parametri che vengono addestrati durante il processo di addestramento del modello per cercare di minimizzare la funzione di loss.

In [0]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 11)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 32)                   384       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 32)                   1056      ['dense[0][0]']               
                                                                                                  
 quality_layer (Dense)       (None, 64)                   2112      ['dense_1[0][0]']             
                                                                                              


La funzione `fit` del modello viene utilizzata per addestrare il modello sui dati di addestramento. Durante il processo di addestramento, il modello cerca di apprendere i pesi e i bias delle connessioni tra i neuroni dei diversi layer del modello in modo da minimizzare la funzione di loss.

Gli step di addestramento in output ci forniscono informazioni sul processo di addestramento del modello. Questi step includono:

- `loss`: il valore della funzione di loss ottenuto dopo l'addestramento del modello sui dati di validazione. Un valore di loss più basso indica una migliore performance del modello.
- `wine_quality_loss`: il valore della funzione di loss specifica per la variabile "qualità del vino" ottenuto dopo l'addestramento del modello sui dati di validazione.
- `wine_type_loss`: il valore della funzione di loss specifica per la variabile "tipo di vino" ottenuto dopo l'addestramento del modello sui dati di validazione.
- `wine_quality_rmse`: la radice dell'errore quadratico medio (RMSE) per la variabile "qualità del vino" ottenuto dopo l'addestramento del modello sui dati di validazione. Un valore di RMSE più basso indica una migliore performance del modello.
- `wine_type_accuracy`: l'accuratezza del modello nella previsione della variabile "tipo di vino" ottenuta dopo l'addestramento del modello sui dati di validazione. Un valore di accuratezza più alto indica una migliore performance del modello.

Questi output ci permettono di valutare la performance del modello dopo l'addestramento e di confrontarla con i risultati ottenuti durante l'addestramento stesso. In questo modo possiamo capire se il modello sta imparando correttamente dai dati di addestramento e se sta generalizzando bene su nuovi dati.

La backpropagation è un algoritmo utilizzato durante la fase di addestramento di una rete neurale per calcolare il gradiente della funzione di loss rispetto ai pesi e ai bias del modello. Questo calcolo del gradiente consente di aggiornare i pesi e i bias in modo da minimizzare la funzione di loss durante l'ottimizzazione del modello.

Durante la fase di forward pass, i dati di addestramento vengono propagati attraverso il modello per ottenere le previsioni del modello. Successivamente, durante la fase di backward pass, il gradiente della funzione di loss viene calcolato rispetto ai pesi e ai bias del modello utilizzando la regola della catena. Questo gradiente viene quindi utilizzato per aggiornare i pesi e i bias del modello utilizzando un algoritmo di ottimizzazione come la discesa del gradiente.

La backpropagation è utile per la fase di fit del modello perché consente di calcolare il gradiente della funzione di loss rispetto ai pesi e ai bias del modello. Questo gradiente viene utilizzato per aggiornare i pesi e i bias del modello in modo da migliorare la performance del modello durante l'addestramento. Senza la backpropagation, sarebbe molto più difficile ottimizzare i pesi e i bias del modello e ottenere una buona performance di addestramento.

In [0]:
log_dir="runs/our_experiment"

history = model.fit(train_X, train_y, 
                    epochs = 40, validation_data=(val_X, val_y), callbacks=[TensorBoard(log_dir=log_dir)])

Epoch 1/40
130/130 [==============================] - 2s 8ms/step - loss: 9.9758 - y_q_layer_loss: 9.4329 - y_t_layer_loss: 0.5429 - y_q_layer_root_mean_squared_error: 3.0713 - y_t_layer_accuracy: 0.7611 - val_loss: 2.5025 - val_y_q_layer_loss: 2.1075 - val_y_t_layer_loss: 0.3950 - val_y_q_layer_root_mean_squared_error: 1.4517 - val_y_t_layer_accuracy: 0.8240
Epoch 2/40
130/130 [==============================] - 0s 2ms/step - loss: 2.0554 - y_q_layer_loss: 1.7703 - y_t_layer_loss: 0.2851 - y_q_layer_root_mean_squared_error: 1.3305 - y_t_layer_accuracy: 0.9127 - val_loss: 1.5229 - val_y_q_layer_loss: 1.3099 - val_y_t_layer_loss: 0.2130 - val_y_q_layer_root_mean_squared_error: 1.1445 - val_y_t_layer_accuracy: 0.9613
Epoch 3/40
130/130 [==============================] - 0s 2ms/step - loss: 1.3358 - y_q_layer_loss: 1.1765 - y_t_layer_loss: 0.1593 - y_q_layer_root_mean_squared_error: 1.0847 - y_t_layer_accuracy: 0.9765 - val_loss: 1.0441 - val_y_q_layer_loss: 0.9180 - val_y_t_layer_loss: 0.

INFO:tensorflow:Assets written to: /local_disk0/repl_tmp_data/ReplId-794e4-84dac-c0983-9/tmpmh5lnxok/model/data/model/assets


Uploading artifacts:   0%|          | 0/12 [00:00<?, ?it/s]

Uploading artifacts:   0%|          | 0/10 [00:00<?, ?it/s]

In [0]:
loss, wine_quality_loss, wine_type_loss, wine_quality_rmse, wine_type_accuracy = model.evaluate(x=val_X, y=val_y)

print()
print(f'loss: {loss}')
print(f'wine_quality_loss: {wine_quality_loss}')
print(f'wine_type_loss: {wine_type_loss}')
print(f'wine_quality_rmse: {wine_quality_rmse}')
print(f'wine_type_accuracy: {wine_type_accuracy}')

33/33 [==============================] - 0s 1ms/step - loss: 0.5163 - y_q_layer_loss: 0.4927 - y_t_layer_loss: 0.0235 - y_q_layer_root_mean_squared_error: 0.7019 - y_t_layer_accuracy: 0.9961

loss: 0.5162505507469177
wine_quality_loss: 0.4927070736885071
wine_type_loss: 0.023543452844023705
wine_quality_rmse: 0.7019309401512146
wine_type_accuracy: 0.9961315393447876



La cella di codice successiva carica l'estensione di TensorBoard per Jupyter Notebook utilizzando il comando `%load_ext tensorboard`. TensorBoard è uno strumento di visualizzazione fornito da TensorFlow che consente di monitorare e analizzare i modelli di machine learning. L'estensione di TensorBoard consente di visualizzare i log di TensorBoard direttamente all'interno di Jupyter Notebook.

L'importazione dell'estensione di TensorBoard è utile quando si desidera utilizzare TensorBoard per visualizzare i log di addestramento e monitorare le prestazioni del modello durante l'addestramento. L'estensione di TensorBoard semplifica l'uso di TensorBoard all'interno di Jupyter Notebook, consentendo di visualizzare i log senza dover aprire un'altra finestra o terminale.

In [0]:
# Import tensorboard logger from PyTorch
#from torch.utils.tensorboard import SummaryWriter

# Load tensorboard extension for Jupyter Notebook, only need to start TB in the notebook
%load_ext tensorboard

2024-10-10 10:02:01.816570: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [0]:
%tensorboard --logdir runs/our_experiment

Your log directory might be ephemeral to the cluster, which will be deleted after cluster termination or restart. You can choose a log directory under /dbfs/ or /Volumes/ to persist your logs in DBFS or UC Volumes.
Tensorboard may not be displayed in the notebook cell output when 'Third-party iFraming prevention' is disabled. You can still use Tensorboard by clicking the link below to open Tensorboard in a new tab. To enable Tensorboard in notebook cell output, please ask your workspace admin to enable 'Third-party iFraming prevention'.


Launching TensorBoard...

Open in a new tab
 
 Note: TensorBoard is only available when this notebook remains attached to the cluster. 
 
 
 Note: This cell needs to be re-run for TensorBoard to be available if this notebook is imported into a different workspace.